In [ ]:
%%capture
! pip install --upgrade --force-reinstall git+https://github.com/huggingface/transformers

In [1]:
%cd MaskDetection/

/content/drive/MyDrive/MaskDetection


In [2]:
import torch
import torchvision.transforms as transforms
import os
import pandas as pd
from skimage import io
from torch.utils.data import (
    Dataset,
    DataLoader
)

from transformers import BeitFeatureExtractor, BeitForImageClassification

model_name_or_path = 'microsoft/beit-base-patch16-224-pt22k'
feature_extractor = BeitFeatureExtractor.from_pretrained(model_name_or_path)

class FeatureExtractor(object):
    def __call__(self, image, target):
        sample = feature_extractor(image, return_tensors='pt')
        sample["labels"] = target
        return sample

class MaskDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))
        
        data = self.transform(image,y_label)

        return data

train_ds = MaskDataset(
    csv_file="train.csv",
    root_dir="",
    # transform=transforms.ToTensor(),
    transform=FeatureExtractor(),
)

test_ds = MaskDataset(
    csv_file="test.csv",
    root_dir="",
    # transform=transforms.ToTensor(),
    transform=FeatureExtractor(),
)

Downloading:   0%|          | 0.00/276 [00:00<?, ?B/s]

In [3]:
train_ds.__getitem__(0)['pixel_values'].shape

torch.Size([1, 3, 224, 224])

In [ ]:
! pip install datasets

In [8]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

In [9]:
labels = ["No Mask","Mask"]

model = BeitForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    ignore_mismatched_sizes=True,
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/351M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at microsoft/beit-base-patch16-224-pt22k were not used when initializing BeitForImageClassification: ['lm_head.weight', 'lm_head.bias', 'layernorm.weight', 'layernorm.bias']
- This IS expected if you are initializing BeitForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BeitForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificatio

In [25]:
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
  output_dir="./beit-tiny-mask-finetuned",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  save_total_limit = 4, # Only last 4 models are saved. Older ones are deleted.
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

PyTorch: setting up devices


In [29]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'][0] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [30]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=feature_extractor,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

Using amp half precision backend


In [31]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14428
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3608


Step,Training Loss,Validation Loss,Accuracy
100,0.497800,0.177015,0.940211
200,0.186300,0.080292,0.974972
300,0.081800,0.206466,0.942436
400,0.198200,0.168230,0.943826
500,0.029300,0.088829,0.982481


***** Running Evaluation *****
  Num examples = 3596
  Batch size = 8
Saving model checkpoint to ./beit-tiny-mask-finetuned/checkpoint-100
Configuration saved in ./beit-tiny-mask-finetuned/checkpoint-100/config.json
Model weights saved in ./beit-tiny-mask-finetuned/checkpoint-100/pytorch_model.bin
Feature extractor saved in ./beit-tiny-mask-finetuned/checkpoint-100/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 3596
  Batch size = 8
Saving model checkpoint to ./beit-tiny-mask-finetuned/checkpoint-200
Configuration saved in ./beit-tiny-mask-finetuned/checkpoint-200/config.json
Model weights saved in ./beit-tiny-mask-finetuned/checkpoint-200/pytorch_model.bin
Feature extractor saved in ./beit-tiny-mask-finetuned/checkpoint-200/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 3596
  Batch size = 8
Saving model checkpoint to ./beit-tiny-mask-finetuned/checkpoint-300
Configuration saved in ./beit-tiny-mask-finetuned/checkpoint-300/config.jso

***** train metrics *****
  epoch                    =        0.55
  total_flos               = 576468516GF
  train_loss               =       0.151
  train_runtime            =  0:58:16.56
  train_samples_per_second =      16.505
  train_steps_per_second   =       1.032


In [32]:
metrics = trainer.evaluate(test_ds)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** Running Evaluation *****
  Num examples = 3596
  Batch size = 8


***** eval metrics *****
  epoch                   =       0.55
  eval_accuracy           =      0.975
  eval_loss               =     0.0803
  eval_runtime            = 0:03:13.02
  eval_samples_per_second =     18.629
  eval_steps_per_second   =      2.331
